In [2]:
#Importing dependencies
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
import os 

In [3]:
#performing data augmentation

train_datagen = ImageDataGenerator(
    rescale=1./255, 
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,  
    shear_range=0.2, 
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',  
)
#scaling the validation data
val_datagen = ImageDataGenerator(rescale=1./255)

#setting the path variables
train_dir = 'data/train/'
val_dir = 'data/test/'

#Setting up the training generator
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),  
    batch_size=32,
    class_mode='categorical', 
)

#Setting up the validation generator
val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
)


Found 18345 images belonging to 10 classes.
Found 4585 images belonging to 10 classes.


In [3]:
#Loading a pretained model (VGG16)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

num_classes = 10

#for not training the base layer
for layer in base_model.layers:
    layer.trainable = False

#setting up out model
x = base_model.output
x = GlobalAveragePooling2D()(x)  
x = Dense(1024, activation='relu')(x) 
predictions = Dense(num_classes, activation='softmax')(x)  

model = Model(inputs=base_model.input, outputs=predictions)

model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [4]:
#training the model

epochs = 10
steps_per_epoch = train_generator.samples // train_generator.batch_size
validation_steps = val_generator.samples // val_generator.batch_size

history = model.fit(
    train_generator,
    epochs=epochs,
    steps_per_epoch=steps_per_epoch,
    validation_data=val_generator,
    validation_steps=validation_steps,
)


Epoch 1/10
573/573 [==============================] - 512s 852ms/step - loss: 1.1742 - accuracy: 0.6133 - val_loss: 0.8108 - val_accuracy: 0.7194
Epoch 2/10
573/573 [==============================] - 214s 372ms/step - loss: 0.7511 - accuracy: 0.7459 - val_loss: 0.7035 - val_accuracy: 0.7537
Epoch 3/10
573/573 [==============================] - 209s 364ms/step - loss: 0.6550 - accuracy: 0.7783 - val_loss: 0.7173 - val_accuracy: 0.7603
Epoch 4/10
573/573 [==============================] - 209s 365ms/step - loss: 0.5938 - accuracy: 0.7941 - val_loss: 0.6479 - val_accuracy: 0.7736
Epoch 5/10
573/573 [==============================] - 218s 379ms/step - loss: 0.5661 - accuracy: 0.8043 - val_loss: 0.5200 - val_accuracy: 0.8142
Epoch 6/10
573/573 [==============================] - 214s 373ms/step - loss: 0.5264 - accuracy: 0.8185 - val_loss: 0.5770 - val_accuracy: 0.7959
Epoch 7/10
573/573 [==============================] - 221s 384ms/step - loss: 0.5208 - accuracy: 0.8219 - val_loss: 0.5322 -

In [5]:
model.save('crop_disease_model.h5')


In [6]:
#further fine-tuning
for layer in base_model.layers[-5:]:
    layer.trainable = True

model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Continue training the model
history_fine = model.fit(
    train_generator,
    epochs=5,  # Fine-tuning for a few more epochs
    steps_per_epoch=steps_per_epoch,
    validation_data=val_generator,
    validation_steps=validation_steps,
)


C:\Users\yksut\AppData\Roaming\Python\Python38\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/5
573/573 [==============================] - 223s 367ms/step - loss: 0.5093 - accuracy: 0.8324 - val_loss: 0.4381 - val_accuracy: 0.8440
Epoch 2/5
573/573 [==============================] - 207s 361ms/step - loss: 0.2532 - accuracy: 0.9115 - val_loss: 0.2095 - val_accuracy: 0.9309
Epoch 3/5
573/573 [==============================] - 204s 356ms/step - loss: 0.1953 - accuracy: 0.9317 - val_loss: 0.2103 - val_accuracy: 0.9255
Epoch 4/5
573/573 [==============================] - 215s 374ms/step - loss: 0.1543 - accuracy: 0.9450 - val_loss: 0.2283 - val_accuracy: 0.9242
Epoch 5/5
573/573 [==============================] - 212s 369ms/step - loss: 0.1393 - accuracy: 0.9511 - val_loss: 0.2589 - val_accuracy: 0.9117


In [7]:
model.save('crop_disease_fine_tuned_model.h5')

In [8]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np

model = load_model('crop_disease_fine_tuned_model.h5')

def predict_disease(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    prediction = model.predict(img_array)
    predicted_class = np.argmax(prediction)
    return predicted_class


In [20]:
index_to_class = {v: k for k, v in train_generator.class_indices.items()}

predicted_class_index = predict_disease('image.jpg')

prediction = index_to_class[predicted_class_index]

print(prediction)


1/1 [==============================] - 0s 44ms/step
Tomato___Early_blight


In [4]:
train_generator.class_indices.items()

dict_items([('Tomato___Bacterial_spot', 0), ('Tomato___Early_blight', 1), ('Tomato___Late_blight', 2), ('Tomato___Leaf_Mold', 3), ('Tomato___Septoria_leaf_spot', 4), ('Tomato___Spider_mites Two-spotted_spider_mite', 5), ('Tomato___Target_Spot', 6), ('Tomato___Tomato_Yellow_Leaf_Curl_Virus', 7), ('Tomato___Tomato_mosaic_virus', 8), ('Tomato___healthy', 9)])